# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report
import pickle

import nltk
nltk.download(['punkt', 'wordnet', 'omw-1.4'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\2100\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\2100\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\2100\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [30]:
# load data from database
engine = create_engine('sqlite:///.\data\DisasterResponse.db')
df = pd.read_sql('SELECT * FROM messages', engine)
#df = df.sample(n=5000, random_state=1)
X = df.message.values
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1).values

In [31]:
df.shape[0]

26177

### 2. Write a tokenization function to process your text data

In [11]:
def tokenize(text):
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [12]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(DecisionTreeClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [32]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [14]:
pipeline.fit(X_train, Y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000287672E7040>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=DecisionTreeClassifier()))])

In [15]:
pickle.dump(pipeline, open('disaster_recovery_pipeline.pkl', 'wb'))

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [16]:
Y_pred = pipeline.predict(X_test)

In [17]:
range_columns = range(len(Y[0])-1)
range_rows_test = range(len(Y_test)-1)
range_rows_pred = range(len(Y_pred)-1)

In [18]:
report = dict.fromkeys(range_columns, [])
for col in range_columns:
    Y_test_col = [Y_test[i][col] for i in range_rows_test]
    Y_pred_col = [Y_pred[i][col] for i in range_rows_pred]
    try:
        target_names = np.unique(Y_test_col).astype('str')
        report[col] = classification_report(Y_test_col, Y_pred_col, target_names=target_names)
    except:
        target_names = np.unique(Y_pred_col).astype('str')
        report[col] = classification_report(Y_test_col, Y_pred_col, target_names=target_names)

In [19]:
for k in report.keys():
    print(report[k])

              precision    recall  f1-score   support

           0       0.45      0.40      0.42       325
           1       0.79      0.83      0.81       917
           2       0.00      0.00      0.00         7

    accuracy                           0.71      1249
   macro avg       0.42      0.41      0.41      1249
weighted avg       0.70      0.71      0.71      1249

              precision    recall  f1-score   support

           0       0.92      0.90      0.91      1054
           1       0.51      0.57      0.54       195

    accuracy                           0.85      1249
   macro avg       0.72      0.73      0.72      1249
weighted avg       0.86      0.85      0.85      1249

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      1246
           1       0.00      0.00      0.00         3

    accuracy                           0.99      1249
   macro avg       0.50      0.50      0.50      1249
weighted avg       1

### 6. Improve your model
Use grid search to find better parameters. 

In [20]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x00000287672E7040>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=DecisionTreeClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x00000287672E7040>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=DecisionTreeClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,


In [22]:
parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'vect__max_df': (0.5, 0.75, 1.0),
        'vect__max_features': (None, 5000, 10000),
        'tfidf__use_idf': (True, False),
        'clf__estimator': [DecisionTreeClassifier(), KNeighborsClassifier()]
    }

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=20)

In [23]:
cv.fit(X_train, Y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
[CV 1/5; 1/72] START clf__estimator=DecisionTreeClassifier(), tfidf__use_idf=True, vect__max_df=0.5, vect__max_features=None, vect__ngram_range=(1, 1)
[CV 1/5; 1/72] END clf__estimator=DecisionTreeClassifier(), tfidf__use_idf=True, vect__max_df=0.5, vect__max_features=None, vect__ngram_range=(1, 1);, score=0.133 total time=  13.3s
[CV 2/5; 1/72] START clf__estimator=DecisionTreeClassifier(), tfidf__use_idf=True, vect__max_df=0.5, vect__max_features=None, vect__ngram_range=(1, 1)
[CV 2/5; 1/72] END clf__estimator=DecisionTreeClassifier(), tfidf__use_idf=True, vect__max_df=0.5, vect__max_features=None, vect__ngram_range=(1, 1);, score=0.129 total time=  13.0s
[CV 3/5; 1/72] START clf__estimator=DecisionTreeClassifier(), tfidf__use_idf=True, vect__max_df=0.5, vect__max_features=None, vect__ngram_range=(1, 1)
[CV 3/5; 1/72] END clf__estimator=DecisionTreeClassifier(), tfidf__use_idf=True, vect__max_df=0.5, vect__max_features=Non

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


[CV 1/5; 37/72] END clf__estimator=KNeighborsClassifier(), tfidf__use_idf=True, vect__max_df=0.5, vect__max_features=None, vect__ngram_range=(1, 1);, score=0.215 total time=   3.8s
[CV 2/5; 37/72] START clf__estimator=KNeighborsClassifier(), tfidf__use_idf=True, vect__max_df=0.5, vect__max_features=None, vect__ngram_range=(1, 1)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


[CV 2/5; 37/72] END clf__estimator=KNeighborsClassifier(), tfidf__use_idf=True, vect__max_df=0.5, vect__max_features=None, vect__ngram_range=(1, 1);, score=0.181 total time=   6.5s
[CV 3/5; 37/72] START clf__estimator=KNeighborsClassifier(), tfidf__use_idf=True, vect__max_df=0.5, vect__max_features=None, vect__ngram_range=(1, 1)


...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


[CV 4/5; 72/72] END clf__estimator=KNeighborsClassifier(), tfidf__use_idf=False, vect__max_df=1.0, vect__max_features=10000, vect__ngram_range=(1, 2);, score=0.144 total time=   2.9s
[CV 5/5; 72/72] START clf__estimator=KNeighborsClassifier(), tfidf__use_idf=False, vect__max_df=1.0, vect__max_features=10000, vect__ngram_range=(1, 2)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


[CV 5/5; 72/72] END clf__estimator=KNeighborsClassifier(), tfidf__use_idf=False, vect__max_df=1.0, vect__max_features=10000, vect__ngram_range=(1, 2);, score=0.159 total time=   2.8s


GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x00000287672E7040>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=DecisionTreeClassifier()))]),
             param_grid={'clf__estimator': [DecisionTreeClassifier(),
                                            KNeighborsClassifier()],
                         'tfidf__use_idf': (True, False),
                         'vect__max_df': (0.5, 0.75, 1.0),
                         'vect__max_features': (None, 5000, 10000),
                         'vect__ngram_range': ((1, 1), (1, 2))},
             verbose=20)

In [24]:
pickle.dump(cv, open('disaster_recovery_gridsearch.pkl', 'wb'))

In [25]:
pickle.dump(cv, open('classifier.pkl', 'wb'))

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [26]:
cv = pickle.load(open('disaster_recovery_gridsearch.pkl', 'rb'))

In [33]:
Y_pred_cv = cv.predict(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [34]:
report_cv = dict.fromkeys(range_columns, [])
for col in range_columns:
    Y_test_col = [Y_test[i][col] for i in range_rows_test]
    Y_pred_col = [Y_pred_cv[i][col] for i in range_rows_pred]
    try:
        target_names = np.unique(Y_test_col).astype('str')
        report_cv[col] = classification_report(Y_test_col, Y_pred_col, target_names=target_names)
    except:
        target_names = np.unique(Y_pred_col).astype('str')
        report_cv[col] = classification_report(Y_test_col, Y_pred_col, target_names=target_names)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metr

In [35]:
for k in report_cv.keys():
    print(report_cv[k])

              precision    recall  f1-score   support

           0       0.77      0.19      0.31       299
           1       0.79      0.98      0.88       945
           2       0.00      0.00      0.00         5

    accuracy                           0.79      1249
   macro avg       0.52      0.39      0.40      1249
weighted avg       0.78      0.79      0.74      1249

              precision    recall  f1-score   support

           0       0.87      0.98      0.92      1026
           1       0.82      0.34      0.48       223

    accuracy                           0.87      1249
   macro avg       0.85      0.66      0.70      1249
weighted avg       0.86      0.87      0.85      1249

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      1240
           1       0.00      0.00      0.00         9

    accuracy                           0.99      1249
   macro avg       0.50      0.50      0.50      1249
weighted avg       0

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
pickle.dump(pipeline, open('disaster_recovery_pipeline.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.